In [3]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
df=pd.read_csv("creditcard.csv")[:80000]
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [4]:
X=df.drop(columns=['Time','Amount','Class']).values
y=df['Class'].values

In [5]:
print(f"{X.shape} {y.shape} {y.sum()}")

(80000, 28) (80000,) 196


In [9]:
def min_recall_precision(y_true, y_pred):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    return min(recall, precision)
make_scorer(min_recall_precision, greater_is_better=False)

NameError: name 'make_scorer' is not defined

In [11]:
from sklearn.linear_model import LogisticRegression
mod=LogisticRegression(class_weight={0:1,1:2},max_iter=1000)
mod.fit(X,y).predict(X).sum()

172

In [12]:
from sklearn.metrics import precision_score,recall_score,make_scorer
precision_score(y,grid.predict(X))
recall_score(y,grid.predict(X))

NameError: name 'grid' is not defined

In [ ]:
from collections import Counter
from sklearn.ensemble import IsolationForest
mod=IsolationForest().fit(X)
mod.predict(X)
Counter(mod.predict(X))
np.where(mod.predict(X)==-1,1,0)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'class_weight': [{0: 1, 1: v} for v in np.linspace(1,20,30)]}
def min_recall_precision(est, X, y_true, sample_weight=None):
    y_pred=est.predict(X)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    return min(recall, precision)
grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    scoring={'precision':make_scorer(precision_score),'recall_score':make_scorer(recall_score),'min_both': min_recall_precision},
    refit='min_both',
    param_grid=param_grid,
    return_train_score=True,
    cv=10,
    n_jobs=-1
)
grid.fit(X, y,sample_weight=np.log(1+df['Amount']))
#make_scorer(min_recall_precision, greater_is_better=False)
# ?make_scorer

In [ ]:
pd.DataFrame(grid.cv_results_)

In [17]:
#lr=LogisticRegression
#??lr.score

In [19]:
#print(grid.cv_results_.keys())


In [21]:
#plt.figure(figsize=(12, 4))
#df_results = pd.DataFrame(grid.cv_results_)
#for score in ['mean_test_recall', 'mean_test_precision', 'mean_test_min_both']:
#    plt.plot([_[1] for _ in df_results['param_class_weight']], 
#             df_results[score], 
#             label=score)
#plt.legend();

In [23]:
s=make_scorer(min_recall_precision)
#??s

In [25]:
def outlier_precision(mod, X, y):
    preds = mod.predict(X)
    return precision_score(y, np.where(preds == -1, 1, 0))

def outlier_recall(mod, X, y):
    preds = mod.predict(X)
    return recall_score(y, np.where(preds == -1, 1, 0))

grid = GridSearchCV(
    estimator=IsolationForest(),
    param_grid={'contamination': np.linspace(0.001, 0.02, 10)},
    scoring={'precision': outlier_precision, 
             'recall': outlier_recall},
    refit='precision',
    cv=5,
    n_jobs=-1
)
grid.fit(X, y);

plt.figure(figsize=(12, 4))
df_results = pd.DataFrame(grid.cv_results_)
for score in ['mean_test_recall', 'mean_test_precision']:
    plt.plot(df_results['param_contamination'], 
             df_results[score], 
             label=score)
plt.legend();

NameError: name 'GridSearchCV' is not defined